In [1]:
#%load_ext autoreload
#%autoreload 2
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler, MinMaxScaler

from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.linear_model import Lasso
import lightgbm as lgb
import tensorflow as tf
import xgboost as xgb

### Import Feathers and Files

In [3]:
#current #1
import feather
#Feathers
X_train = pd.read_feather(r'C:\Users\user\Desktop\Kaggle\HousePrices\Feathers\X_train_FE_D_13.feather')
test = pd.read_feather(r'C:\Users\user\Desktop\Kaggle\HousePrices\Feathers\test_FE_D_13.feather')

#csv
y_temp = pd.read_csv(r'C:\Users\user\Desktop\Kaggle\HousePrices\Feathers\y_train_FE_D_13.csv', low_memory = False, header = None, names = 'y')
y_train = y_temp['y']

test_ID_temp = pd.read_csv(r'C:\Users\user\Desktop\Kaggle\HousePrices\Feathers\test_ID.csv', low_memory = False, header = None, names = 'i')
test_ID = test_ID_temp['i']

print(X_train.shape,"  ",test.shape,"  ",y_train.shape, "  ", test_ID.shape)

(1453, 312)    (1459, 312)    (1453,)    (1459,)


# New best, submission_42

import feather
#Feathers
X_train = pd.read_feather(r'C:\Users\user\Desktop\Kaggle\HousePrices\Feathers\X_train_FE_D_11.feather')
test = pd.read_feather(r'C:\Users\user\Desktop\Kaggle\HousePrices\Feathers\test_FE_D_11.feather')

#csv
y_temp = pd.read_csv(r'C:\Users\user\Desktop\Kaggle\HousePrices\Feathers\y_train_FE_D_11.csv', low_memory = False, header = None, names = 'y')
y_train = y_temp['y']

test_ID_temp = pd.read_csv(r'C:\Users\user\Desktop\Kaggle\HousePrices\Feathers\test_ID.csv', low_memory = False, header = None, names = 'i')
test_ID = test_ID_temp['i']

print(X_train.shape,"  ",test.shape,"  ",y_train.shape, "  ", test_ID.shape)

## MLXTEND

In [4]:
# mlextnd

from mlxtend.regressor import StackingCVRegressor

lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=4321))

GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05, max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, loss='huber', random_state =5)

model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,learning_rate=0.05, n_estimators=720, max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319, feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11, n_jobs = 1)

model_xgb =  xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, learning_rate=0.05, max_depth=3, min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571, subsample=0.5213, silent=1,random_state =7, nthread = -1, n_jobs = 1)

model_xgb_2 = xgb.XGBRegressor(learning_rate =0.01, n_estimators=3460, max_depth=3,min_child_weight=0 ,
                                     gamma=0, subsample=0.7,colsample_bytree=0.7,objective= 'reg:linear',nthread=4,
                                     scale_pos_weight=1,seed=27, reg_alpha=0.00006)

model_xgb_3 = make_pipeline(RobustScaler(), xgb.XGBRegressor( colsample_bytree=0.2, gamma=0.0, learning_rate=0.01, max_depth=4,
                     min_child_weight=1.5, n_estimators=7200, reg_alpha=0.9, reg_lambda=0.6, subsample=0.2, seed=4321,
                     silent=1))

RF_model = RandomForestRegressor(n_estimators=500, min_samples_leaf=1, max_features = 0.5, n_jobs = 1, 
                                 oob_score = True, random_state = 4321)

#stack
stack_gen = StackingCVRegressor(regressors = (lasso, GBoost, model_xgb_2, model_lgb), meta_regressor = lasso,
                                            use_features_in_secondary=True) #use_features_in_secondary=True uses original dat and results

#prepare dataframes
stack_X = np.array(X_train)
stack_y = np.array(y_train)

In [5]:
mlxt_model = stack_gen.fit(stack_X, stack_y)

In [6]:
#modified for stack_gen
n_folds = 5

def rmsle_cv_stack(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    rmse= np.sqrt(-cross_val_score(model, stack_X, stack_y, scoring="neg_mean_squared_error", cv = kf, verbose = 2))
    return(rmse)

In [7]:
#stack_gen CV
score = rmsle_cv_stack(mlxt_model)
print("\nStack score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

[CV]  ................................................................
[CV] ................................................. , total= 3.5min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.5min remaining:    0.0s


[CV] ................................................. , total= 3.4min
[CV]  ................................................................
[CV] ................................................. , total= 3.5min
[CV]  ................................................................
[CV] ................................................. , total= 3.5min
[CV]  ................................................................
[CV] ................................................. , total= 3.4min

Stack score: 0.1019 (0.0057)



[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 17.3min finished


#### Stacking Regressor

In [ ]:
from mlxtend.regressor import StackingRegressor

stack_2_gen = StackingRegressor(regressors = (lasso, GBoost, model_xgb_2, model_lgb), meta_regressor = lasso)

#prepare dataframes
stack_X = np.array(X_train)
stack_y = np.array(y_train)

In [ ]:
mlxt_2_model = stack_2_gen.fit(stack_X, stack_y)

In [ ]:
#stack_2_gen CV
score = rmsle_cv_stack(mlxt_2_model)
print("\nStack_2 score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
#Sound******************************
import winsound
duration = 500  # millisecond
freq = 500
freq_2 = 450  # Hz
winsound.Beep(freq, duration)
winsound.Beep(freq_2, duration)
winsound.Beep(freq, duration)

## Feature Importance MLExtnd

#### view features

In [ ]:
FI_mlxt = pd.DataFrame({"Feature Importance":mlxt_2_model.coef_}, index = X_train.columns)
FI_mlxt.sort_values("Feature Importance",ascending=False)

In [ ]:
FI_mlxt[FI_mlxt["Feature Importance"]!=0].sort_values("Feature Importance").plot(kind="barh",figsize=(15,25))
plt.xticks(rotation=90)
plt.show()

#### Reduce features

In [ ]:
def mlxt_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.coef_}).sort_values('imp', ascending=False)

In [ ]:
fi = mlxt_feat_importance(mlxt_2_model, X_train)
fi[:10]

In [ ]:
to_keep = fi[fi.imp>0.0000001].cols
len(to_keep)
to_keep = to_keep.append(fi[fi.imp<-0.0000001].cols)
len(to_keep)

In [ ]:
equal_zero = fi[fi.imp == 0].cols
len(equal_zero)

In [ ]:
X_train = X_train[to_keep].copy()
test = test[to_keep].copy()
X_train.shape
test.shape

### Cross Validation

In [ ]:
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    rmse= np.sqrt(-cross_val_score(model, X_train.values, y_train, scoring="neg_mean_squared_error", cv = kf, verbose = 2))
    return(rmse)

In [ ]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=4321))
#lasso = Lasso(alpha =0.0005, random_state=4321)

%time lasso.fit(X_train, y_train)

In [ ]:
#LASSO CV
score = rmsle_cv(lasso)
print("\nLASSO score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

## Stacked Models mlens Superlearner

In [ ]:
# mlens
from mlens.ensemble import SuperLearner
from mlens.metrics.metrics import rmse

ensemble = SuperLearner(scorer=rmse, random_state=4321, verbose=1, backend = 'multiprocessing', folds=10, n_jobs = 1)

In [ ]:
ensemble.add([ GBoost, model_lgb, lasso], backend = 'multiprocessing', folds=5, n_jobs = 1)
ensemble.add_meta(lasso, backend = 'multiprocessing', folds=10)
ensemble_model = ensemble.fit(X_train, y_train, backend = 'multiprocessing', folds=10) 

In [ ]:
score = rmsle_cv(ensemble)

print("\n Ensemble score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

#### Normalized Weight *Convert to function later

In [ ]:
kaggle_score = pd.DataFrame()

kaggle_score['Score'] = [1-.14039, 1-.12425, 1-.12494, 1-.12218, 1-.1253, 1-.11862]
#print(kaggle_score)

scaler = MinMaxScaler(feature_range=(0.1, .99))
scaler.fit(kaggle_score)
#scaler.transform(kaggle_score)

kaggle_min_max = scaler.fit_transform(kaggle_score)
#print(kaggle_min_max)
kaggle_score['min_max']= kaggle_min_max
kaggle_score['Model'] = ['RF','Lasso','GBoost','LGBM', 'XGB', 'meta']
kaggle_score['weight']= (kaggle_score['min_max']/(kaggle_score['min_max'].sum(axis = 0)))*100
print(kaggle_score)

# Predictions

In [ ]:
light_gbm_pred = np.expm1(model_lgb.predict(test.values))

In [ ]:
XGB_pred = np.expm1(model_xgb.predict(test))  #remove .values for xgb

In [ ]:
#weighted_pred = (light_gbm_pred *0.40 +Lasso_pred * 0.25 + GBoost_pred * 0.25 + RF_pred * 0.10) *****Best***

In [ ]:
#weighted_pred_2 = (light_gbm_pred *0.27 +Lasso_pred * 0.24 + GBoost_pred * 0.23+ XGB_pred * 0.23 + RF_pred * 0.03)

In [ ]:
weighted_pred_meta = (ensemble_pred * 0.26 + light_gbm_pred *0.21 +Lasso_pred * 0.18 + GBoost_pred * 0.18+ XGB_pred * 0.17)

In [ ]:
averaged_pred = np.expm1(averaged_models.predict(test.values))

In [ ]:
ensemble_pred = np.expm1(ensemble_model.predict(test.values))

In [8]:
mlxt_pred = np.expm1(mlxt_model.predict(test.values))   #****MLXT*****

In [ ]:
stack_avg_pred = (mlxt_pred * 0.50 + ensemble_pred *0.50)

# Final Submission

In [9]:
sub_1 = pd.DataFrame()
sub_1['Id'] = test_ID
sub_1['SalePrice'] = mlxt_pred
sub_1.to_csv('submission_44.csv',index=False)